In [226]:
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np 

In [228]:
X, y = make_classification(n_samples=200, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

In [230]:
class MyKNNClf: 
    def __init__(self, k=3, metric='euclidean'): 
        self.k = k
        self.train_size = None
        self.metric = metric

        self.__valid()
    
    def __repr__(self): 
        return f'MyKNNClf class: k={self.k}'
        
    def __valid(self): 
        if self.metric not in ['euclidean', 'chebyshev', 'manhattan', 'cosine']:
            raise ValueError(f"Invalid metric '{self.metric}'. Choose from: 'euclidean', 'chebyshev', 'manhattan', 'cosine'.")
                               

    def __metric(self, X_train :pd.DataFrame, X_test: pd.DataFrame):
        '''Этот метод принимает тренеровачные и тестовые данные и на них считает метрики'''
        if self.metric == 'euclidean':
            dist = np.sqrt(np.sum((X_train-X_test)**2, axis=1))
            return dist 
        if self.metric == 'chebyshev': 
            dist = np.max(np.abs(X_train - X_test), axis=1)
            return dist 
        if self.metric == 'manhattan':
            dist = np.sum(np.abs(X_train - X_test), axis=1)
            return dist 
        if self.metric == 'cosine': 
            dist = 1 - (np.dot(X_train, X_test) /
            (np.linalg.norm(X_train, axis=1) * np.linalg.norm(X_test)))
            return dist
        
    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.__X_train, self.__y_train = X.values, y.values 
        self.train_size = X.shape

    def predict(self, X: pd.DataFrame): 
        X_test = X.values
        preds = [] 
        for x in X_test: # проходимся по каждому объекту из тестовой выборки 
            # расстояние до всех объектов train
            dist = self.__metric(self.__X_train, x)
            # индексы k ближайших 
            k_idx = np.argsort(dist)[:self.k] # сортируем по возростанию 
            k_labels = self.__y_train[k_idx]  # находим лейблы самых близких точек
            counts = np.bincount(k_labels,  minlength=2)    # считаем сколько раз встречается каждый класс
            if counts[0] > counts[1]: # если кол 0 больше 1
                preds.append(0)
            else: 
                preds.append(1)
        return np.array(preds)

    def predict_proba(self, X: pd.DataFrame): 
        X_test = X.values
        probs = [] 
        for x in X_test: # проходимся по каждому объекту из тестовой выборки 
            # расстояние от x до всех объектов из train 
            dist = self.__metric(self.__X_train, x)
            k_idx = np.argsort(dist)[:self.k]
            k_labels = self.__y_train[k_idx]
            counts = np.bincount(k_labels, minlength=2)
            probs.append(counts[1]/self.k) # вероятность принадлежности к 1ому классу
        return np.array(probs)
            

    


In [232]:
model = MyKNNClf(metric='manhattan')

In [234]:
model.fit(X, y)

In [236]:
model.predict(X)

array([1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1])

In [238]:
a = np.array([[4, 2, 5, 3], 
            [4, 12, 3, 12]]
            )
b = np.array([[3, 1, 3, 1],
            [4, 2, 5, 3]])

In [240]:
np.abs(a-b)

array([[ 1,  1,  2,  2],
       [ 0, 10,  2,  9]])

In [242]:
np.sum(np.abs(a-b), axis=0)

array([ 1, 11,  4, 11])

In [244]:
a = np.array([3, 4])


In [246]:
np.linalg.norm(a)

5.0

In [252]:
model.predict_proba(X)

array([1.        , 1.        , 0.        , 0.33333333, 1.        ,
       1.        , 0.        , 0.        , 0.33333333, 1.        ,
       0.        , 1.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 1.        ,
       0.        , 0.        , 0.        , 0.66666667, 0.        ,
       1.        , 1.        , 1.        , 0.        , 0.        ,
       1.        , 1.        , 1.        , 0.        , 1.        ,
       1.        , 0.        , 0.        , 1.        , 1.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.        , 1.        , 1.        , 1.        , 0.66666667,
       1.        , 1.        , 0.        , 1.        , 1.        ,
       1.        , 1.        , 0.        , 0.        , 0.        ,
       1.        , 0.66666667, 1.        , 1.        , 0.        ,
       0.        , 0.        , 0.33333333, 0.        , 0.        ,
       1.        , 0.        , 1.        , 0.        , 0.     